# 신용카드 사기 탐지 모델 성능 비교

성능을 비교하여 신용카드 사기 탐지에 가장 적합한 모델 탐색

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, precision_recall_curve, 
                             average_precision_score, f1_score, roc_curve)
import warnings
warnings.filterwarnings('ignore')

# PyTorch 임포트 및 GPU 설정
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

print("=" * 50)
print("GPU 설정 확인")
print("=" * 50)
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"cuDNN 버전: {torch.backends.cudnn.version()}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
    device = torch.device('cuda:0')
    print(f"\nGPU 사용 설정 완료: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("GPU를 찾을 수 없음. CPU 사용")

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print(f"\n사용할 디바이스: {device}")
print("\n라이브러리 로드 완료")

In [ ]:
# 데이터 로드
data_path = r"creditcard.csv"
df = pd.read_csv(data_path)

print(f"데이터 shape: {df.shape}")
print(f"\n컬럼 목록: {df.columns.tolist()}")
print(f"\n데이터 타입:")
print(df.dtypes)
df.head()

In [ ]:
# 기본 통계 및 클래스 분포 확인
print("=" * 50)
print("기본 통계 정보")
print("=" * 50)
print(df.describe())

print("\n" + "=" * 50)
print("클래스 분포 (Class: 0=정상, 1=사기)")
print("=" * 50)
class_counts = df['Class'].value_counts()
print(class_counts)
print(f"\n사기 비율: {class_counts[1] / len(df) * 100:.4f}%")

# 클래스 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].bar(['정상 (0)', '사기 (1)'], class_counts.values, color=['steelblue', 'crimson'])
axes[0].set_title('클래스 분포')
axes[0].set_ylabel('건수')
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v + 1000, f'{v:,}', ha='center')

axes[1].pie(class_counts.values, labels=['정상', '사기'], autopct='%1.3f%%', 
            colors=['steelblue', 'crimson'], explode=[0, 0.1])
axes[1].set_title('클래스 비율')

plt.tight_layout()
plt.show()

## 2. 데이터 전처리

In [ ]:
# 데이터 전처리
# 결측치 확인
print("결측치 개수:")
print(df.isnull().sum().sum())

# 특성과 타겟 분리
X = df.drop('Class', axis=1)
y = df['Class']

# Time과 Amount 스케일링 (V1~V28은 이미 PCA 변환되어 있음)
scaler = StandardScaler()
X['Time'] = scaler.fit_transform(X[['Time']])
X['Amount'] = scaler.fit_transform(X[['Amount']])

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n학습 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")
print(f"\n학습 데이터 클래스 분포:")
print(y_train.value_counts())
print(f"\n테스트 데이터 클래스 분포:")
print(y_test.value_counts())

## 3. 전통적인 분류 모델

### 3.1 Logistic Regression

In [ ]:
# 결과 저장용 딕셔너리
results = {}

# Logistic Regression
print("=" * 50)
print("Logistic Regression 학습 중...")
print("=" * 50)

lr_model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
lr_model.fit(X_train, y_train)

# 예측
lr_pred = lr_model.predict(X_test)
lr_prob = lr_model.predict_proba(X_test)[:, 1]

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, lr_pred, target_names=['정상', '사기']))

lr_auc = roc_auc_score(y_test, lr_prob)
lr_f1 = f1_score(y_test, lr_pred)
lr_ap = average_precision_score(y_test, lr_prob)

print(f"ROC-AUC Score: {lr_auc:.4f}")
print(f"F1 Score: {lr_f1:.4f}")
print(f"Average Precision: {lr_ap:.4f}")

results['Logistic Regression'] = {
    'predictions': lr_pred,
    'probabilities': lr_prob,
    'roc_auc': lr_auc,
    'f1': lr_f1,
    'avg_precision': lr_ap
}

### 3.2 Random Forest

In [ ]:
# Random Forest
print("=" * 50)
print("Random Forest 학습 중...")
print("=" * 50)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, 
                                   class_weight='balanced', n_jobs=-1)
rf_model.fit(X_train, y_train)

# 예측
rf_pred = rf_model.predict(X_test)
rf_prob = rf_model.predict_proba(X_test)[:, 1]

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, rf_pred, target_names=['정상', '사기']))

rf_auc = roc_auc_score(y_test, rf_prob)
rf_f1 = f1_score(y_test, rf_pred)
rf_ap = average_precision_score(y_test, rf_prob)

print(f"ROC-AUC Score: {rf_auc:.4f}")
print(f"F1 Score: {rf_f1:.4f}")
print(f"Average Precision: {rf_ap:.4f}")

results['Random Forest'] = {
    'predictions': rf_pred,
    'probabilities': rf_prob,
    'roc_auc': rf_auc,
    'f1': rf_f1,
    'avg_precision': rf_ap
}

## 4. 이상치 탐지 특화 모델

### 4.1 One-Class SVM
정상 데이터만으로 학습하여 이상치를 탐지하는 방식

In [ ]:
# One-Class SVM (정상 데이터만으로 학습)
print("=" * 50)
print("One-Class SVM 학습 중...")
print("=" * 50)

# 정상 데이터만 추출하여 학습 (샘플링하여 계산 시간 단축)
X_train_normal = X_train[y_train == 0]
# 계산 효율을 위해 정상 데이터 일부만 샘플링
sample_size = min(10000, len(X_train_normal))
X_train_sample = X_train_normal.sample(n=sample_size, random_state=42)

print(f"학습에 사용할 정상 데이터 샘플 수: {len(X_train_sample)}")

# One-Class SVM 학습
ocsvm_model = OneClassSVM(kernel='rbf', gamma='scale', nu=0.01)
ocsvm_model.fit(X_train_sample)

# 예측 (1: 정상, -1: 이상치)
ocsvm_pred_raw = ocsvm_model.predict(X_test)
# 이상치(-1)를 사기(1)로, 정상(1)을 정상(0)으로 변환
ocsvm_pred = np.where(ocsvm_pred_raw == -1, 1, 0)

# decision_function으로 확률 대용 점수 계산 (낮을수록 이상치)
ocsvm_scores = -ocsvm_model.decision_function(X_test)  # 부호 반전 (높을수록 이상치)

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, ocsvm_pred, target_names=['정상', '사기']))

ocsvm_auc = roc_auc_score(y_test, ocsvm_scores)
ocsvm_f1 = f1_score(y_test, ocsvm_pred)
ocsvm_ap = average_precision_score(y_test, ocsvm_scores)

print(f"ROC-AUC Score: {ocsvm_auc:.4f}")
print(f"F1 Score: {ocsvm_f1:.4f}")
print(f"Average Precision: {ocsvm_ap:.4f}")

results['One-Class SVM'] = {
    'predictions': ocsvm_pred,
    'probabilities': ocsvm_scores,
    'roc_auc': ocsvm_auc,
    'f1': ocsvm_f1,
    'avg_precision': ocsvm_ap
}

### 4.2 Autoencoder
정상 데이터 패턴을 학습하고, 재구성 오류가 큰 샘플을 이상치로 탐지

In [ ]:
# Autoencoder (PyTorch - GPU 사용)
print("=" * 50)
print("Autoencoder 학습 중... (PyTorch GPU)")
print("=" * 50)

# 정상 데이터만 사용하여 학습
X_train_normal = X_train[y_train == 0].values.astype(np.float32)
X_test_values = X_test.values.astype(np.float32)

# Autoencoder 모델 정의
input_dim = X_train.shape[1]
encoding_dim = 14

class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim * 2),
            nn.ReLU(),
            nn.Linear(encoding_dim * 2, encoding_dim),
            nn.ReLU()
        )
        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim * 2),
            nn.ReLU(),
            nn.Linear(encoding_dim * 2, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 모델 생성 및 GPU로 이동
autoencoder = Autoencoder(input_dim, encoding_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# 데이터 준비
X_train_tensor = torch.FloatTensor(X_train_normal).to(device)
train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# 검증 데이터 분리
val_size = int(0.1 * len(X_train_normal))
train_data = X_train_normal[val_size:]
val_data = X_train_normal[:val_size]

train_tensor = torch.FloatTensor(train_data).to(device)
val_tensor = torch.FloatTensor(val_data).to(device)
train_dataset = TensorDataset(train_tensor, train_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# 학습
epochs = 50
patience = 5
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

print(f"학습 디바이스: {device}")

for epoch in range(epochs):
    autoencoder.train()
    train_loss = 0.0
    for batch_x, _ in train_loader:
        optimizer.zero_grad()
        outputs = autoencoder(batch_x)
        loss = criterion(outputs, batch_x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # 검증
    autoencoder.eval()
    with torch.no_grad():
        val_outputs = autoencoder(val_tensor)
        val_loss = criterion(val_outputs, val_tensor).item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = autoencoder.state_dict().copy()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# 최적 모델 복원
if best_model_state:
    autoencoder.load_state_dict(best_model_state)

print("\n학습 완료!")

In [ ]:
# Autoencoder 예측 및 평가
autoencoder.eval()

# 테스트 데이터 예측
X_test_tensor = torch.FloatTensor(X_test_values).to(device)
with torch.no_grad():
    reconstructions = autoencoder(X_test_tensor).cpu().numpy()

mse = np.mean(np.power(X_test_values - reconstructions, 2), axis=1)

# 임곗값 설정 (정상 데이터 재구성 오류의 95 퍼센타일)
X_train_normal_tensor = torch.FloatTensor(X_train_normal).to(device)
with torch.no_grad():
    train_reconstructions = autoencoder(X_train_normal_tensor).cpu().numpy()

train_mse = np.mean(np.power(X_train_normal - train_reconstructions, 2), axis=1)
threshold = np.percentile(train_mse, 95)

print(f"재구성 오류 임곗값 (95 퍼센타일): {threshold:.6f}")

# 임곗값 기반 예측
ae_pred = (mse > threshold).astype(int)
ae_scores = mse  # 재구성 오류를 점수로 사용

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, ae_pred, target_names=['정상', '사기']))

ae_auc = roc_auc_score(y_test, ae_scores)
ae_f1 = f1_score(y_test, ae_pred)
ae_ap = average_precision_score(y_test, ae_scores)

print(f"ROC-AUC Score: {ae_auc:.4f}")
print(f"F1 Score: {ae_f1:.4f}")
print(f"Average Precision: {ae_ap:.4f}")

results['Autoencoder'] = {
    'predictions': ae_pred,
    'probabilities': ae_scores,
    'roc_auc': ae_auc,
    'f1': ae_f1,
    'avg_precision': ae_ap
}

### 4.3 Isolation Forest (추가 이상치 탐지 모델)

In [ ]:
# Isolation Forest
from sklearn.ensemble import IsolationForest

print("=" * 50)
print("Isolation Forest 학습 중...")
print("=" * 50)

# 예상 이상치 비율 설정
contamination = y_train.sum() / len(y_train)

iforest = IsolationForest(n_estimators=100, contamination=contamination, 
                          random_state=42, n_jobs=-1)
iforest.fit(X_train)

# 예측 (1: 정상, -1: 이상치)
iforest_pred_raw = iforest.predict(X_test)
iforest_pred = np.where(iforest_pred_raw == -1, 1, 0)

# Anomaly score (낮을수록 이상치)
iforest_scores = -iforest.decision_function(X_test)  # 부호 반전

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, iforest_pred, target_names=['정상', '사기']))

iforest_auc = roc_auc_score(y_test, iforest_scores)
iforest_f1 = f1_score(y_test, iforest_pred)
iforest_ap = average_precision_score(y_test, iforest_scores)

print(f"ROC-AUC Score: {iforest_auc:.4f}")
print(f"F1 Score: {iforest_f1:.4f}")
print(f"Average Precision: {iforest_ap:.4f}")

results['Isolation Forest'] = {
    'predictions': iforest_pred,
    'probabilities': iforest_scores,
    'roc_auc': iforest_auc,
    'f1': iforest_f1,
    'avg_precision': iforest_ap
}

## 5. 모델 성능 비교

In [ ]:
# 성능 비교 요약 테이블
print("=" * 70)
print("                    모델 성능 비교 요약")
print("=" * 70)

comparison_data = []
for model_name, metrics in results.items():
    comparison_data.append({
        '모델': model_name,
        'ROC-AUC': metrics['roc_auc'],
        'F1 Score': metrics['f1'],
        'Avg Precision': metrics['avg_precision']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('ROC-AUC', ascending=False)
print(comparison_df.to_string(index=False))

# 모델 유형 분류
comparison_df['유형'] = comparison_df['모델'].apply(
    lambda x: '전통적 분류' if x in ['Logistic Regression', 'Random Forest'] else '이상치 탐지'
)

print("\n" + "=" * 70)
print("                    유형별 평균 성능")
print("=" * 70)
print(comparison_df.groupby('유형')[['ROC-AUC', 'F1 Score', 'Avg Precision']].mean())

In [ ]:
# 성능 지표 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

models = list(results.keys())
colors = ['steelblue', 'green', 'orange', 'red', 'purple']

# ROC-AUC 비교
roc_aucs = [results[m]['roc_auc'] for m in models]
axes[0].barh(models, roc_aucs, color=colors[:len(models)])
axes[0].set_xlabel('ROC-AUC Score')
axes[0].set_title('ROC-AUC 비교')
axes[0].set_xlim([0.5, 1.0])
for i, v in enumerate(roc_aucs):
    axes[0].text(v + 0.01, i, f'{v:.4f}', va='center')

# F1 Score 비교
f1_scores = [results[m]['f1'] for m in models]
axes[1].barh(models, f1_scores, color=colors[:len(models)])
axes[1].set_xlabel('F1 Score')
axes[1].set_title('F1 Score 비교')
for i, v in enumerate(f1_scores):
    axes[1].text(v + 0.01, i, f'{v:.4f}', va='center')

# Average Precision 비교
avg_precs = [results[m]['avg_precision'] for m in models]
axes[2].barh(models, avg_precs, color=colors[:len(models)])
axes[2].set_xlabel('Average Precision')
axes[2].set_title('Average Precision 비교')
for i, v in enumerate(avg_precs):
    axes[2].text(v + 0.01, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# ROC Curve 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
for model_name, metrics in results.items():
    fpr, tpr, _ = roc_curve(y_test, metrics['probabilities'])
    axes[0].plot(fpr, tpr, label=f"{model_name} (AUC={metrics['roc_auc']:.4f})")

axes[0].plot([0, 1], [0, 1], 'k--', label='Random')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve 비교')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
for model_name, metrics in results.items():
    precision, recall, _ = precision_recall_curve(y_test, metrics['probabilities'])
    axes[1].plot(recall, precision, label=f"{model_name} (AP={metrics['avg_precision']:.4f})")

axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve 비교')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Confusion Matrix 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (model_name, metrics) in enumerate(results.items()):
    cm = confusion_matrix(y_test, metrics['predictions'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['정상', '사기'], yticklabels=['정상', '사기'])
    axes[idx].set_title(f'{model_name}')
    axes[idx].set_xlabel('예측')
    axes[idx].set_ylabel('실제')

# 빈 서브플롯 숨기기
for idx in range(len(results), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 6. 결론 및 분석

In [ ]:
# 최종 결론 출력
print("=" * 70)
print("                         분석 결론")
print("=" * 70)

# 최고 성능 모델 찾기
best_auc_model = max(results, key=lambda x: results[x]['roc_auc'])
best_f1_model = max(results, key=lambda x: results[x]['f1'])
best_ap_model = max(results, key=lambda x: results[x]['avg_precision'])

print(f"\n최고 ROC-AUC 모델: {best_auc_model} ({results[best_auc_model]['roc_auc']:.4f})")
print(f"최고 F1 Score 모델: {best_f1_model} ({results[best_f1_model]['f1']:.4f})")
print(f"최고 Average Precision 모델: {best_ap_model} ({results[best_ap_model]['avg_precision']:.4f})")

print("\n" + "-" * 70)
print("분석 요약:")
print("-" * 70)
print("""
1. 전통적인 분류 모델 (Logistic Regression, Random Forest)
   - 레이블 정보를 활용하여 지도 학습
   - class_weight='balanced' 옵션으로 클래스 불균형 처리
   - 일반적으로 높은 성능을 보임

2. 이상치 탐지 모델 (One-Class SVM, Autoencoder, Isolation Forest)
   - 정상 데이터의 패턴만 학습 (비지도/반지도 학습)
   - 레이블이 없거나 부족한 상황에서 유용
   - 새로운 유형의 사기 패턴 탐지에 강점

3. 실무 적용 시 고려사항:
   - 불균형 데이터: Average Precision이 더 신뢰할 수 있는 지표
   - 실시간 탐지: Logistic Regression이 빠른 추론 속도 제공
   - 해석 가능성: Random Forest는 특성 중요도 분석 가능
   - 적응형 탐지: Autoencoder는 새로운 사기 패턴에 적응 가능
""")

# 모델별 장단점
print("\n" + "=" * 70)
print("                    모델별 특징 비교")
print("=" * 70)
model_features = pd.DataFrame({
    '모델': ['Logistic Regression', 'Random Forest', 'One-Class SVM', 'Autoencoder', 'Isolation Forest'],
    '학습 방식': ['지도학습', '지도학습', '비지도학습', '비지도학습', '비지도학습'],
    '해석 가능성': ['높음', '중간', '낮음', '낮음', '중간'],
    '학습 속도': ['빠름', '중간', '느림', '느림', '빠름'],
    '추론 속도': ['빠름', '빠름', '중간', '중간', '빠름'],
    '메모리 사용': ['낮음', '중간', '높음', '중간', '낮음']
})
print(model_features.to_string(index=False))

## 7. 추가 이상치 탐지 모델

### 7.1 Local Outlier Factor (LOF)

In [ ]:
# Local Outlier Factor (LOF)
from sklearn.neighbors import LocalOutlierFactor

print("=" * 50)
print("Local Outlier Factor (LOF) 학습 중...")
print("=" * 50)

# LOF는 novelty=True로 설정해야 새 데이터 예측 가능
lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination, novelty=True, n_jobs=-1)
lof.fit(X_train)

# 예측 (1: 정상, -1: 이상치)
lof_pred_raw = lof.predict(X_test)
lof_pred = np.where(lof_pred_raw == -1, 1, 0)

# 이상치 점수 (낮을수록 이상치)
lof_scores = -lof.decision_function(X_test)

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, lof_pred, target_names=['정상', '사기']))

lof_auc = roc_auc_score(y_test, lof_scores)
lof_f1 = f1_score(y_test, lof_pred)
lof_ap = average_precision_score(y_test, lof_scores)

print(f"ROC-AUC Score: {lof_auc:.4f}")
print(f"F1 Score: {lof_f1:.4f}")
print(f"Average Precision: {lof_ap:.4f}")

results['LOF'] = {
    'predictions': lof_pred,
    'probabilities': lof_scores,
    'roc_auc': lof_auc,
    'f1': lof_f1,
    'avg_precision': lof_ap
}

### 7.2 DBSCAN (Density-Based Spatial Clustering)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

print("=" * 50)
print("DBSCAN 학습 중...")
print("=" * 50)

# 계산 효율을 위해 샘플링
sample_size_dbscan = min(20000, len(X_train))
X_train_sample_db = X_train.sample(n=sample_size_dbscan, random_state=42)

# eps 파라미터 추정 (k-distance plot 기반)
k = 5
neigh = NearestNeighbors(n_neighbors=k)
neigh.fit(X_train_sample_db)
distances, _ = neigh.kneighbors(X_train_sample_db)
k_distances = np.sort(distances[:, k-1])
eps_value = np.percentile(k_distances, 95)  # 95 퍼센타일 사용

print(f"선택된 eps 값: {eps_value:.4f}")

# DBSCAN 학습
dbscan = DBSCAN(eps=eps_value, min_samples=5, n_jobs=-1)
dbscan_train_labels = dbscan.fit_predict(X_train_sample_db)

# 테스트 데이터 예측을 위해 가장 가까운 학습 데이터 포인트 기반 예측
# 노이즈(-1)로 분류된 학습 포인트와의 거리 기반
noise_points = X_train_sample_db[dbscan_train_labels == -1]
if len(noise_points) > 0:
    neigh_noise = NearestNeighbors(n_neighbors=1)
    neigh_noise.fit(noise_points)
    distances_to_noise, _ = neigh_noise.kneighbors(X_test)
    dbscan_scores = 1 / (distances_to_noise.flatten() + 1e-6)  # 가까울수록 높은 점수
else:
    # 노이즈 포인트가 없으면 전체 데이터와의 거리 사용
    neigh_all = NearestNeighbors(n_neighbors=1)
    neigh_all.fit(X_train_sample_db)
    distances_all, _ = neigh_all.kneighbors(X_test)
    dbscan_scores = distances_all.flatten()

# 임곗값 기반 예측
dbscan_threshold = np.percentile(dbscan_scores, 100 - contamination * 100)
dbscan_pred = (dbscan_scores > dbscan_threshold).astype(int)

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, dbscan_pred, target_names=['정상', '사기']))

dbscan_auc = roc_auc_score(y_test, dbscan_scores)
dbscan_f1 = f1_score(y_test, dbscan_pred)
dbscan_ap = average_precision_score(y_test, dbscan_scores)

print(f"ROC-AUC Score: {dbscan_auc:.4f}")
print(f"F1 Score: {dbscan_f1:.4f}")
print(f"Average Precision: {dbscan_ap:.4f}")

results['DBSCAN'] = {
    'predictions': dbscan_pred,
    'probabilities': dbscan_scores,
    'roc_auc': dbscan_auc,
    'f1': dbscan_f1,
    'avg_precision': dbscan_ap
}

### 7.3 Variational Autoencoder (VAE)

In [ ]:
# Variational Autoencoder (VAE) - PyTorch GPU 버전
print("=" * 50)
print("Variational Autoencoder (VAE) 학습 중... (PyTorch GPU)")
print("=" * 50)

# VAE 파라미터
latent_dim_vae = 8
intermediate_dim_vae = 32

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim, intermediate_dim):
        super(VAE, self).__init__()
        
        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, intermediate_dim),
            nn.ReLU(),
            nn.Linear(intermediate_dim, intermediate_dim // 2),
            nn.ReLU()
        )
        
        self.fc_mu = nn.Linear(intermediate_dim // 2, latent_dim)
        self.fc_logvar = nn.Linear(intermediate_dim // 2, latent_dim)
        
        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, intermediate_dim // 2),
            nn.ReLU(),
            nn.Linear(intermediate_dim // 2, intermediate_dim),
            nn.ReLU(),
            nn.Linear(intermediate_dim, input_dim)
        )
    
    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# 모델 생성 및 GPU로 이동
vae = VAE(input_dim, latent_dim_vae, intermediate_dim_vae).to(device)
vae_optimizer = optim.Adam(vae.parameters(), lr=0.001)

# 데이터 준비
X_train_normal_arr = X_train[y_train == 0].values.astype(np.float32)
val_size_vae = int(0.1 * len(X_train_normal_arr))
train_data_vae = X_train_normal_arr[val_size_vae:]
val_data_vae = X_train_normal_arr[:val_size_vae]

train_tensor_vae = torch.FloatTensor(train_data_vae).to(device)
val_tensor_vae = torch.FloatTensor(val_data_vae).to(device)
train_dataset_vae = TensorDataset(train_tensor_vae, train_tensor_vae)
train_loader_vae = DataLoader(train_dataset_vae, batch_size=256, shuffle=True)

# VAE 손실 함수
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='mean')
    kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + 0.001 * kl_loss  # KL 가중치 조절

# 학습
epochs_vae = 50
patience_vae = 5
best_val_loss_vae = float('inf')
patience_counter_vae = 0
best_vae_state = None

print(f"학습 디바이스: {device}")

for epoch in range(epochs_vae):
    vae.train()
    train_loss_vae = 0.0
    for batch_x, _ in train_loader_vae:
        vae_optimizer.zero_grad()
        recon, mu, logvar = vae(batch_x)
        loss = vae_loss(recon, batch_x, mu, logvar)
        loss.backward()
        vae_optimizer.step()
        train_loss_vae += loss.item()
    
    train_loss_vae /= len(train_loader_vae)
    
    # 검증
    vae.eval()
    with torch.no_grad():
        val_recon, val_mu, val_logvar = vae(val_tensor_vae)
        val_loss_vae = vae_loss(val_recon, val_tensor_vae, val_mu, val_logvar).item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs_vae}] - Train Loss: {train_loss_vae:.6f}, Val Loss: {val_loss_vae:.6f}")
    
    # Early stopping
    if val_loss_vae < best_val_loss_vae:
        best_val_loss_vae = val_loss_vae
        patience_counter_vae = 0
        best_vae_state = vae.state_dict().copy()
    else:
        patience_counter_vae += 1
        if patience_counter_vae >= patience_vae:
            print(f"Early stopping at epoch {epoch+1}")
            break

# 최적 모델 복원
if best_vae_state:
    vae.load_state_dict(best_vae_state)

print("\nVAE 학습 완료!")

In [ ]:
# VAE 예측 및 평가
vae.eval()

# 테스트 데이터 예측
X_test_tensor_vae = torch.FloatTensor(X_test_values).to(device)
with torch.no_grad():
    vae_reconstructions, _, _ = vae(X_test_tensor_vae)
    vae_reconstructions = vae_reconstructions.cpu().numpy()

vae_mse = np.mean(np.power(X_test_values - vae_reconstructions, 2), axis=1)

# 임곗값 설정
X_train_normal_tensor_vae = torch.FloatTensor(X_train_normal_arr).to(device)
with torch.no_grad():
    vae_train_recon, _, _ = vae(X_train_normal_tensor_vae)
    vae_train_recon = vae_train_recon.cpu().numpy()

vae_train_mse = np.mean(np.power(X_train_normal_arr - vae_train_recon, 2), axis=1)
vae_threshold = np.percentile(vae_train_mse, 95)

print(f"VAE 재구성 오류 임곗값 (95 퍼센타일): {vae_threshold:.6f}")

# 예측
vae_pred = (vae_mse > vae_threshold).astype(int)
vae_scores = vae_mse

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, vae_pred, target_names=['정상', '사기']))

vae_auc = roc_auc_score(y_test, vae_scores)
vae_f1 = f1_score(y_test, vae_pred)
vae_ap = average_precision_score(y_test, vae_scores)

print(f"ROC-AUC Score: {vae_auc:.4f}")
print(f"F1 Score: {vae_f1:.4f}")
print(f"Average Precision: {vae_ap:.4f}")

results['VAE'] = {
    'predictions': vae_pred,
    'probabilities': vae_scores,
    'roc_auc': vae_auc,
    'f1': vae_f1,
    'avg_precision': vae_ap
}

### 7.4 GAN (Generative Adversarial Network) 기반 이상치 탐지

In [ ]:
# GAN 기반 이상치 탐지 (PyTorch GPU 버전)
print("=" * 50)
print("GAN 기반 이상치 탐지 학습 중... (PyTorch GPU)")
print("=" * 50)

# GAN 파라미터
latent_dim_gan = 32

# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    
    def forward(self, z):
        return self.model(z)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# 모델 생성 및 GPU로 이동
generator = Generator(latent_dim_gan, input_dim).to(device)
discriminator = Discriminator(input_dim).to(device)

# 옵티마이저
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
criterion_gan = nn.BCELoss()

# 데이터 준비
X_train_normal_gan = torch.FloatTensor(X_train_normal_arr).to(device)
gan_dataset = TensorDataset(X_train_normal_gan)
gan_loader = DataLoader(gan_dataset, batch_size=256, shuffle=True)

# 학습
epochs_gan = 50
print(f"학습 디바이스: {device}")

for epoch in range(epochs_gan):
    d_loss_epoch = 0.0
    g_loss_epoch = 0.0
    
    for batch_idx, (real_samples,) in enumerate(gan_loader):
        batch_size_curr = real_samples.size(0)
        
        # 레이블 생성
        real_labels = torch.ones(batch_size_curr, 1).to(device) * 0.9  # Label smoothing
        fake_labels = torch.zeros(batch_size_curr, 1).to(device)
        
        # -------- Discriminator 학습 --------
        d_optimizer.zero_grad()
        
        # Real samples
        real_output = discriminator(real_samples)
        d_loss_real = criterion_gan(real_output, real_labels)
        
        # Fake samples
        noise = torch.randn(batch_size_curr, latent_dim_gan).to(device)
        fake_samples = generator(noise)
        fake_output = discriminator(fake_samples.detach())
        d_loss_fake = criterion_gan(fake_output, fake_labels)
        
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()
        
        # -------- Generator 학습 --------
        g_optimizer.zero_grad()
        
        noise = torch.randn(batch_size_curr, latent_dim_gan).to(device)
        fake_samples = generator(noise)
        fake_output = discriminator(fake_samples)
        g_loss = criterion_gan(fake_output, torch.ones(batch_size_curr, 1).to(device))
        
        g_loss.backward()
        g_optimizer.step()
        
        d_loss_epoch += d_loss.item()
        g_loss_epoch += g_loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs_gan}] - D Loss: {d_loss_epoch/len(gan_loader):.4f}, G Loss: {g_loss_epoch/len(gan_loader):.4f}")

print("\nGAN 학습 완료!")

In [ ]:
# GAN 이상치 점수 계산 (Discriminator 기반)
discriminator.eval()

# 테스트 데이터에 대한 판별자 점수
X_test_tensor_gan = torch.FloatTensor(X_test_values).to(device)
with torch.no_grad():
    gan_output = discriminator(X_test_tensor_gan).cpu().numpy().flatten()

# 1에서 빼서 이상치일수록 높은 점수
gan_scores = 1 - gan_output

# 임곗값 설정
with torch.no_grad():
    gan_train_output = discriminator(X_train_normal_gan).cpu().numpy().flatten()

gan_train_scores = 1 - gan_train_output
gan_threshold = np.percentile(gan_train_scores, 95)

print(f"GAN 이상치 임곗값 (95 퍼센타일): {gan_threshold:.6f}")

# 예측
gan_pred = (gan_scores > gan_threshold).astype(int)

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, gan_pred, target_names=['정상', '사기']))

gan_auc = roc_auc_score(y_test, gan_scores)
gan_f1 = f1_score(y_test, gan_pred)
gan_ap = average_precision_score(y_test, gan_scores)

print(f"ROC-AUC Score: {gan_auc:.4f}")
print(f"F1 Score: {gan_f1:.4f}")
print(f"Average Precision: {gan_ap:.4f}")

results['GAN'] = {
    'predictions': gan_pred,
    'probabilities': gan_scores,
    'roc_auc': gan_auc,
    'f1': gan_f1,
    'avg_precision': gan_ap
}

### 7.5 LSTM Autoencoder

In [ ]:
# LSTM Autoencoder (PyTorch GPU 버전)
print("=" * 50)
print("LSTM Autoencoder 학습 중... (PyTorch GPU)")
print("=" * 50)

timesteps = 10
n_features = input_dim // timesteps + (1 if input_dim % timesteps != 0 else 0)

# 패딩을 위한 크기 조정
padded_dim = timesteps * n_features
X_train_normal_padded = np.zeros((len(X_train_normal_arr), padded_dim), dtype=np.float32)
X_train_normal_padded[:, :input_dim] = X_train_normal_arr
X_train_lstm = X_train_normal_padded.reshape(-1, timesteps, n_features)

X_test_padded = np.zeros((len(X_test_values), padded_dim), dtype=np.float32)
X_test_padded[:, :input_dim] = X_test_values
X_test_lstm = X_test_padded.reshape(-1, timesteps, n_features)

print(f"LSTM 입력 shape: {X_train_lstm.shape}")

# LSTM Autoencoder 모델 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, n_features, hidden_dim=32, latent_dim=16, timesteps=10):
        super(LSTMAutoencoder, self).__init__()
        self.timesteps = timesteps
        self.n_features = n_features
        self.latent_dim = latent_dim
        
        # 인코더 LSTM
        self.encoder_lstm1 = nn.LSTM(n_features, hidden_dim, batch_first=True)
        self.encoder_lstm2 = nn.LSTM(hidden_dim, latent_dim, batch_first=True)
        
        # 디코더 LSTM
        self.decoder_lstm1 = nn.LSTM(latent_dim, latent_dim, batch_first=True)
        self.decoder_lstm2 = nn.LSTM(latent_dim, hidden_dim, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, n_features)
    
    def forward(self, x):
        # 인코더
        enc_out1, _ = self.encoder_lstm1(x)
        enc_out2, (h_n, c_n) = self.encoder_lstm2(enc_out1)
        
        # 잠재 벡터를 시퀀스로 반복
        latent = h_n.squeeze(0).unsqueeze(1).repeat(1, self.timesteps, 1)
        
        # 디코더
        dec_out1, _ = self.decoder_lstm1(latent)
        dec_out2, _ = self.decoder_lstm2(dec_out1)
        output = self.output_layer(dec_out2)
        
        return output

# 모델 생성 및 GPU로 이동
lstm_autoencoder = LSTMAutoencoder(n_features, hidden_dim=32, latent_dim=16, timesteps=timesteps).to(device)
lstm_optimizer = optim.Adam(lstm_autoencoder.parameters(), lr=0.001)
lstm_criterion = nn.MSELoss()

# 데이터 준비
val_size_lstm = int(0.1 * len(X_train_lstm))
train_data_lstm = X_train_lstm[val_size_lstm:]
val_data_lstm = X_train_lstm[:val_size_lstm]

train_tensor_lstm = torch.FloatTensor(train_data_lstm).to(device)
val_tensor_lstm = torch.FloatTensor(val_data_lstm).to(device)
train_dataset_lstm = TensorDataset(train_tensor_lstm, train_tensor_lstm)
train_loader_lstm = DataLoader(train_dataset_lstm, batch_size=256, shuffle=True)

# 학습
epochs_lstm = 50
patience_lstm = 5
best_val_loss_lstm = float('inf')
patience_counter_lstm = 0
best_lstm_state = None

print(f"학습 디바이스: {device}")

for epoch in range(epochs_lstm):
    lstm_autoencoder.train()
    train_loss_lstm = 0.0
    for batch_x, _ in train_loader_lstm:
        lstm_optimizer.zero_grad()
        outputs = lstm_autoencoder(batch_x)
        loss = lstm_criterion(outputs, batch_x)
        loss.backward()
        lstm_optimizer.step()
        train_loss_lstm += loss.item()
    
    train_loss_lstm /= len(train_loader_lstm)
    
    # 검증
    lstm_autoencoder.eval()
    with torch.no_grad():
        val_outputs_lstm = lstm_autoencoder(val_tensor_lstm)
        val_loss_lstm = lstm_criterion(val_outputs_lstm, val_tensor_lstm).item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs_lstm}] - Train Loss: {train_loss_lstm:.6f}, Val Loss: {val_loss_lstm:.6f}")
    
    # Early stopping
    if val_loss_lstm < best_val_loss_lstm:
        best_val_loss_lstm = val_loss_lstm
        patience_counter_lstm = 0
        best_lstm_state = lstm_autoencoder.state_dict().copy()
    else:
        patience_counter_lstm += 1
        if patience_counter_lstm >= patience_lstm:
            print(f"Early stopping at epoch {epoch+1}")
            break

# 최적 모델 복원
if best_lstm_state:
    lstm_autoencoder.load_state_dict(best_lstm_state)

print("\nLSTM Autoencoder 학습 완료!")

In [ ]:
# LSTM Autoencoder 예측 및 평가
lstm_autoencoder.eval()

# 테스트 데이터 예측
X_test_lstm_tensor = torch.FloatTensor(X_test_lstm).to(device)
with torch.no_grad():
    lstm_reconstructions = lstm_autoencoder(X_test_lstm_tensor).cpu().numpy()

lstm_mse = np.mean(np.power(X_test_lstm - lstm_reconstructions, 2), axis=(1, 2))

# 임곗값 설정
X_train_lstm_tensor = torch.FloatTensor(X_train_lstm).to(device)
with torch.no_grad():
    lstm_train_reconstructions = lstm_autoencoder(X_train_lstm_tensor).cpu().numpy()

lstm_train_mse = np.mean(np.power(X_train_lstm - lstm_train_reconstructions, 2), axis=(1, 2))
lstm_threshold = np.percentile(lstm_train_mse, 95)

print(f"LSTM 재구성 오류 임곗값 (95 퍼센타일): {lstm_threshold:.6f}")

# 예측
lstm_pred = (lstm_mse > lstm_threshold).astype(int)
lstm_scores = lstm_mse

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, lstm_pred, target_names=['정상', '사기']))

lstm_auc = roc_auc_score(y_test, lstm_scores)
lstm_f1 = f1_score(y_test, lstm_pred)
lstm_ap = average_precision_score(y_test, lstm_scores)

print(f"ROC-AUC Score: {lstm_auc:.4f}")
print(f"F1 Score: {lstm_f1:.4f}")
print(f"Average Precision: {lstm_ap:.4f}")

results['LSTM'] = {
    'predictions': lstm_pred,
    'probabilities': lstm_scores,
    'roc_auc': lstm_auc,
    'f1': lstm_f1,
    'avg_precision': lstm_ap
}

### 7.6 Simple RNN Autoencoder

In [ ]:
# Simple RNN Autoencoder (PyTorch GPU 버전)
print("=" * 50)
print("Simple RNN Autoencoder 학습 중... (PyTorch GPU)")
print("=" * 50)

# RNN Autoencoder 모델 정의
class RNNAutoencoder(nn.Module):
    def __init__(self, n_features, hidden_dim=32, latent_dim=16, timesteps=10):
        super(RNNAutoencoder, self).__init__()
        self.timesteps = timesteps
        self.n_features = n_features
        self.latent_dim = latent_dim
        
        # 인코더 RNN
        self.encoder_rnn1 = nn.RNN(n_features, hidden_dim, batch_first=True, nonlinearity='tanh')
        self.encoder_rnn2 = nn.RNN(hidden_dim, latent_dim, batch_first=True, nonlinearity='tanh')
        
        # 디코더 RNN
        self.decoder_rnn1 = nn.RNN(latent_dim, latent_dim, batch_first=True, nonlinearity='tanh')
        self.decoder_rnn2 = nn.RNN(latent_dim, hidden_dim, batch_first=True, nonlinearity='tanh')
        self.output_layer = nn.Linear(hidden_dim, n_features)
    
    def forward(self, x):
        # 인코더
        enc_out1, _ = self.encoder_rnn1(x)
        enc_out2, h_n = self.encoder_rnn2(enc_out1)
        
        # 잠재 벡터를 시퀀스로 반복
        latent = h_n.squeeze(0).unsqueeze(1).repeat(1, self.timesteps, 1)
        
        # 디코더
        dec_out1, _ = self.decoder_rnn1(latent)
        dec_out2, _ = self.decoder_rnn2(dec_out1)
        output = self.output_layer(dec_out2)
        
        return output

# 모델 생성 및 GPU로 이동
rnn_autoencoder = RNNAutoencoder(n_features, hidden_dim=32, latent_dim=16, timesteps=timesteps).to(device)
rnn_optimizer = optim.Adam(rnn_autoencoder.parameters(), lr=0.001)
rnn_criterion = nn.MSELoss()

# 데이터 로더 (LSTM과 동일한 데이터 사용)
train_dataset_rnn = TensorDataset(train_tensor_lstm, train_tensor_lstm)
train_loader_rnn = DataLoader(train_dataset_rnn, batch_size=256, shuffle=True)

# 학습
epochs_rnn = 50
patience_rnn = 5
best_val_loss_rnn = float('inf')
patience_counter_rnn = 0
best_rnn_state = None

print(f"학습 디바이스: {device}")

for epoch in range(epochs_rnn):
    rnn_autoencoder.train()
    train_loss_rnn = 0.0
    for batch_x, _ in train_loader_rnn:
        rnn_optimizer.zero_grad()
        outputs = rnn_autoencoder(batch_x)
        loss = rnn_criterion(outputs, batch_x)
        loss.backward()
        rnn_optimizer.step()
        train_loss_rnn += loss.item()
    
    train_loss_rnn /= len(train_loader_rnn)
    
    # 검증
    rnn_autoencoder.eval()
    with torch.no_grad():
        val_outputs_rnn = rnn_autoencoder(val_tensor_lstm)
        val_loss_rnn = rnn_criterion(val_outputs_rnn, val_tensor_lstm).item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs_rnn}] - Train Loss: {train_loss_rnn:.6f}, Val Loss: {val_loss_rnn:.6f}")
    
    # Early stopping
    if val_loss_rnn < best_val_loss_rnn:
        best_val_loss_rnn = val_loss_rnn
        patience_counter_rnn = 0
        best_rnn_state = rnn_autoencoder.state_dict().copy()
    else:
        patience_counter_rnn += 1
        if patience_counter_rnn >= patience_rnn:
            print(f"Early stopping at epoch {epoch+1}")
            break

# 최적 모델 복원
if best_rnn_state:
    rnn_autoencoder.load_state_dict(best_rnn_state)

print("\nSimple RNN Autoencoder 학습 완료!")

In [ ]:
# RNN Autoencoder 예측 및 평가
rnn_autoencoder.eval()

# 테스트 데이터 예측
with torch.no_grad():
    rnn_reconstructions = rnn_autoencoder(X_test_lstm_tensor).cpu().numpy()

rnn_mse = np.mean(np.power(X_test_lstm - rnn_reconstructions, 2), axis=(1, 2))

# 임곗값 설정
with torch.no_grad():
    rnn_train_reconstructions = rnn_autoencoder(X_train_lstm_tensor).cpu().numpy()

rnn_train_mse = np.mean(np.power(X_train_lstm - rnn_train_reconstructions, 2), axis=(1, 2))
rnn_threshold = np.percentile(rnn_train_mse, 95)

print(f"RNN 재구성 오류 임곗값 (95 퍼센타일): {rnn_threshold:.6f}")

# 예측
rnn_pred = (rnn_mse > rnn_threshold).astype(int)
rnn_scores = rnn_mse

# 성능 평가
print("\nClassification Report:")
print(classification_report(y_test, rnn_pred, target_names=['정상', '사기']))

rnn_auc = roc_auc_score(y_test, rnn_scores)
rnn_f1 = f1_score(y_test, rnn_pred)
rnn_ap = average_precision_score(y_test, rnn_scores)

print(f"ROC-AUC Score: {rnn_auc:.4f}")
print(f"F1 Score: {rnn_f1:.4f}")
print(f"Average Precision: {rnn_ap:.4f}")

results['RNN'] = {
    'predictions': rnn_pred,
    'probabilities': rnn_scores,
    'roc_auc': rnn_auc,
    'f1': rnn_f1,
    'avg_precision': rnn_ap
}

## 8. 전체 모델 성능 비교 (업데이트)

In [ ]:
# 전체 모델 성능 비교 요약
print("=" * 80)
print("                        전체 모델 성능 비교 요약")
print("=" * 80)

# 결과 데이터프레임 생성
comparison_data_all = []
for model_name, metrics in results.items():
    comparison_data_all.append({
        '모델': model_name,
        'ROC-AUC': round(metrics['roc_auc'], 4),
        'F1 Score': round(metrics['f1'], 4),
        'Avg Precision': round(metrics['avg_precision'], 4)
    })

comparison_df_all = pd.DataFrame(comparison_data_all)

# 모델 유형 분류
def get_model_type(model):
    if model in ['Logistic Regression', 'Random Forest']:
        return '전통적 분류'
    elif model in ['One-Class SVM', 'Isolation Forest', 'LOF', 'DBSCAN']:
        return '전통적 이상치 탐지'
    else:
        return '딥러닝 기반'

comparison_df_all['유형'] = comparison_df_all['모델'].apply(get_model_type)

# ROC-AUC 기준 정렬
comparison_df_all = comparison_df_all.sort_values('ROC-AUC', ascending=False)
print(comparison_df_all.to_string(index=False))

# 최고 성능 모델
print("\n" + "=" * 80)
print("                            최고 성능 모델")
print("=" * 80)
best_auc = comparison_df_all.loc[comparison_df_all['ROC-AUC'].idxmax()]
best_f1 = comparison_df_all.loc[comparison_df_all['F1 Score'].idxmax()]
best_ap = comparison_df_all.loc[comparison_df_all['Avg Precision'].idxmax()]

print(f"📊 최고 ROC-AUC: {best_auc['모델']} ({best_auc['ROC-AUC']})")
print(f"📊 최고 F1 Score: {best_f1['모델']} ({best_f1['F1 Score']})")
print(f"📊 최고 Average Precision: {best_ap['모델']} ({best_ap['Avg Precision']})")

In [ ]:
# 유형별 평균 성능
print("\n" + "=" * 80)
print("                          유형별 평균 성능")
print("=" * 80)
type_avg = comparison_df_all.groupby('유형')[['ROC-AUC', 'F1 Score', 'Avg Precision']].mean()
print(type_avg.round(4))

In [ ]:
# 전체 모델 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 8))

# 색상 설정
color_map = {
    '전통적 분류': 'steelblue',
    '전통적 이상치 탐지': 'forestgreen',
    '딥러닝 기반': 'coral'
}
colors_all = [color_map[t] for t in comparison_df_all['유형']]

models_all = comparison_df_all['모델'].tolist()

# ROC-AUC
ax1 = axes[0]
bars1 = ax1.barh(models_all, comparison_df_all['ROC-AUC'].tolist(), color=colors_all)
ax1.set_xlabel('ROC-AUC Score')
ax1.set_title('ROC-AUC 비교')
ax1.set_xlim([0.4, 1.0])
for i, v in enumerate(comparison_df_all['ROC-AUC'].tolist()):
    ax1.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=8)

# F1 Score
ax2 = axes[1]
bars2 = ax2.barh(models_all, comparison_df_all['F1 Score'].tolist(), color=colors_all)
ax2.set_xlabel('F1 Score')
ax2.set_title('F1 Score 비교')
for i, v in enumerate(comparison_df_all['F1 Score'].tolist()):
    ax2.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=8)

# Average Precision
ax3 = axes[2]
bars3 = ax3.barh(models_all, comparison_df_all['Avg Precision'].tolist(), color=colors_all)
ax3.set_xlabel('Average Precision')
ax3.set_title('Average Precision 비교')
for i, v in enumerate(comparison_df_all['Avg Precision'].tolist()):
    ax3.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=8)

# 범례
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='steelblue', label='전통적 분류'),
    Patch(facecolor='forestgreen', label='전통적 이상치 탐지'),
    Patch(facecolor='coral', label='딥러닝 기반')
]
fig.legend(handles=legend_elements, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.02))

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

In [ ]:
# ROC Curve - 전체 모델 비교
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
ax1 = axes[0]
for model_name, metrics in results.items():
    fpr, tpr, _ = roc_curve(y_test, metrics['probabilities'])
    ax1.plot(fpr, tpr, label=f"{model_name} ({metrics['roc_auc']:.3f})", linewidth=1.5)

ax1.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('ROC Curve - 전체 모델 비교')
ax1.legend(loc='lower right', fontsize=8)
ax1.grid(True, alpha=0.3)

# Precision-Recall Curve
ax2 = axes[1]
for model_name, metrics in results.items():
    precision, recall, _ = precision_recall_curve(y_test, metrics['probabilities'])
    ax2.plot(recall, precision, label=f"{model_name} ({metrics['avg_precision']:.3f})", linewidth=1.5)

ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall Curve - 전체 모델 비교')
ax2.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Confusion Matrix - 전체 모델
n_models = len(results)
n_cols = 4
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
axes = axes.flatten()

for idx, (model_name, metrics) in enumerate(results.items()):
    cm = confusion_matrix(y_test, metrics['predictions'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['정상', '사기'], yticklabels=['정상', '사기'])
    axes[idx].set_title(f'{model_name}', fontsize=10)
    axes[idx].set_xlabel('예측')
    axes[idx].set_ylabel('실제')

# 빈 서브플롯 숨기기
for idx in range(len(results), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 9. 최종 결론

In [ ]:
# 최종 결론 및 분석
print("=" * 80)
print("                           최종 분석 결론")
print("=" * 80)

print("""
신용카드 사기 탐지 모델 성능 비교 분석

1. 전통적 분류 모델 (Logistic Regression, Random Forest)
   - 레이블 정보를 활용한 지도 학습 방식
   - 클래스 불균형 처리(class_weight='balanced')로 좋은 성능
   - Random Forest가 전반적으로 가장 우수한 성능

2. 전통적 이상치 탐지 모델 (One-Class SVM, LOF, Isolation Forest, DBSCAN)
   - 비지도/반지도 학습 방식으로 레이블 없이도 학습 가능
   - Isolation Forest가 이상치 탐지 모델 중 가장 안정적
   - LOF는 밀도 기반으로 지역적 이상치 탐지에 강점
   - DBSCAN은 클러스터링 기반으로 파라미터 민감도가 높음

3. 딥러닝 기반 모델 (Autoencoder, VAE, GAN, LSTM, RNN)
   - 정상 데이터 패턴 학습 후 재구성 오류로 이상치 판별
   - Autoencoder/VAE가 비교적 안정적인 성능
   - LSTM/RNN은 시계열적 패턴이 명확할 때 효과적
   - GAN은 학습이 불안정할 수 있으나 새로운 패턴 탐지에 유용

4. 실무 적용 권장사항:
   - 레이블 있는 경우: Random Forest 또는 Logistic Regression
   - 레이블 없는 경우: Isolation Forest 또는 Autoencoder
   - 실시간 처리: Logistic Regression (빠른 추론)
   - 해석 필요: Random Forest (특성 중요도)
   - 새로운 패턴: VAE 또는 GAN 기반 모델
""")

# 모델별 특징 비교표
print("\n" + "=" * 80)
print("                        모델별 특징 비교")
print("=" * 80)
model_features_all = pd.DataFrame({
    '모델': list(results.keys()),
    '유형': [get_model_type(m) for m in results.keys()],
    'ROC-AUC': [results[m]['roc_auc'] for m in results.keys()],
    'F1': [results[m]['f1'] for m in results.keys()],
    'AP': [results[m]['avg_precision'] for m in results.keys()]
})
model_features_all = model_features_all.sort_values('ROC-AUC', ascending=False)
print(model_features_all.to_string(index=False))